In [ ]:
# Curso: Ciência da Computação
# Disciplina: Inteligência Artificial
# Estudo Dirigido - Trabalho II - Labirinto - Buscas com informação (Gulosa, A *)

# Busca sem informação

from pprint import pprint
from copy import deepcopy


class Ponto:

    def __init__(self, x=0, y=0):
        self.x = x
        self.y = y

    def __repr__(self):
        return 'x{}, y{}'.format(self.x, self.y)

    def set_x(self, x):
        self.x = x

    def set_y(self, y):
        self.y = y


class Estado:
    def __init__(self, vetor=None, custo_atual=0, acao=None, pai=None):
        if vetor is None:
            vetor = []
        self.vetor = vetor
        self.custo_atual = custo_atual
        self.acao = acao
        self.pai = pai

    def __getitem__(self, i):
        return self.vetor[i]

    def __eq__(self, outro):
        return self.vetor == outro

    def __repr__(self):
        print('Caminho: ')
        for linha in self.vetor:
            for coluna in linha:
                print(coluna, end='')
            print()
        return "Ação: {}, Custo Atual: {}, {}".format(self.acao, self.custo_atual, self.pai)

    def get_custo(self):
        return self.custo_atual


def funcao_avaliacao(p1: Ponto, p2: Ponto):
    # heuristica(extrapolação) - custo estimativo de n
    # utilização da distância de manhattan para validar a heuristica(extrapolação)
    return abs(p1.x - p2.x) + abs(p1.y - p2.y)


class Problema:

    @property
    def estado_inicial(self):
        with open('maze01.txt') as fp:
            dados = fp.readlines()

        labirinto = []

        for linha in dados:
            aux = []
            for coluna in linha.strip():
                if coluna == ' ':
                    aux.append(0)
                elif coluna == 'E':
                    aux.append(1)
                elif coluna == 'S':
                    aux.append(2)
                else:
                    aux.append(9)
            labirinto.append(aux)

        p1 = Ponto(20, 31)
        p2 = Ponto()

        for i, linha in enumerate(labirinto):
            for j, coluna in enumerate(linha):
                if linha[j] == 1:
                    p2.set_y(i + 1)
                    p2.set_x(j + 1)

        ponto_atual = funcao_avaliacao(p1, p2)

        return Estado(labirinto, ponto_atual)

    @staticmethod
    def estado_objetivo(estado):
        aux = Estado(estado)
        objetivo = Estado([
            [9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9],
            [0, 0, 0, 0, 0, 0, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 9, 0, 0, 0, 0, 0, 9, 0, 0, 0, 0, 0, 9],
            [9, 0, 0, 9, 0, 0, 9, 0, 0, 9, 9, 9, 9, 9, 9, 9, 0, 0, 9, 9, 9, 9, 0, 0, 9, 0, 0, 9, 0, 0, 9],
            [9, 0, 0, 9, 0, 0, 9, 0, 0, 9, 0, 0, 0, 0, 0, 9, 0, 0, 9, 0, 0, 0, 0, 0, 0, 0, 0, 9, 0, 0, 9],
            [9, 0, 0, 9, 0, 0, 9, 9, 9, 9, 0, 0, 9, 0, 0, 9, 0, 0, 9, 0, 0, 9, 0, 0, 9, 9, 9, 9, 9, 9, 9],
            [9, 0, 0, 9, 0, 0, 0, 0, 0, 0, 0, 0, 9, 0, 0, 0, 0, 0, 0, 0, 0, 9, 0, 0, 9, 0, 0, 0, 0, 0, 9],
            [9, 0, 0, 9, 9, 9, 9, 0, 0, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 0, 0, 9, 0, 0, 9, 0, 0, 9],
            [9, 0, 0, 0, 0, 0, 9, 0, 0, 9, 0, 0, 0, 0, 0, 0, 0, 0, 9, 0, 0, 0, 0, 0, 9, 0, 0, 9, 0, 0, 9],
            [9, 9, 9, 9, 0, 0, 9, 0, 0, 9, 0, 0, 9, 9, 9, 9, 0, 0, 9, 0, 0, 9, 9, 9, 9, 0, 0, 9, 0, 0, 9],
            [9, 0, 0, 9, 0, 0, 9, 0, 0, 0, 0, 0, 9, 0, 0, 0, 0, 0, 9, 0, 0, 0, 0, 0, 9, 0, 0, 9, 0, 0, 9],
            [9, 0, 0, 9, 0, 0, 9, 9, 9, 9, 9, 9, 9, 0, 0, 9, 9, 9, 9, 9, 9, 9, 0, 0, 9, 9, 9, 9, 0, 0, 9],
            [9, 0, 0, 0, 0, 0, 9, 0, 0, 0, 0, 0, 0, 0, 0, 9, 0, 0, 0, 0, 0, 9, 0, 0, 0, 0, 0, 0, 0, 0, 9],
            [9, 9, 9, 9, 9, 9, 9, 0, 0, 9, 9, 9, 9, 9, 9, 9, 0, 0, 9, 0, 0, 9, 9, 9, 9, 0, 0, 9, 9, 9, 9],
            [9, 0, 0, 0, 0, 0, 9, 0, 0, 9, 0, 0, 0, 0, 0, 9, 0, 0, 9, 0, 0, 0, 0, 0, 9, 0, 0, 0, 0, 0, 9],
            [9, 9, 9, 9, 0, 0, 9, 0, 0, 9, 9, 9, 9, 0, 0, 9, 0, 0, 9, 9, 9, 9, 0, 0, 9, 9, 9, 9, 9, 9, 9],
            [9, 0, 0, 0, 0, 0, 9, 0, 0, 9, 0, 0, 0, 0, 0, 0, 0, 0, 9, 0, 0, 9, 0, 0, 0, 0, 0, 0, 0, 0, 9],
            [9, 0, 0, 9, 9, 9, 9, 0, 0, 9, 0, 0, 9, 9, 9, 9, 9, 9, 9, 0, 0, 9, 9, 9, 9, 9, 9, 9, 0, 0, 9],
            [9, 0, 0, 0, 0, 0, 0, 0, 0, 9, 0, 0, 9, 0, 0, 0, 0, 0, 0, 0, 0, 9, 0, 0, 0, 0, 0, 0, 0, 0, 9],
            [9, 0, 0, 9, 0, 0, 9, 9, 9, 9, 0, 0, 9, 0, 0, 9, 9, 9, 9, 9, 9, 9, 0, 0, 9, 9, 9, 9, 9, 9, 9],
            [9, 0, 0, 9, 0, 0, 0, 0, 0, 0, 0, 0, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
            [9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9]])
        return aux == objetivo

    @staticmethod
    def solucao(estado):
        resultado = []
        ptr = estado
        while not ptr.pai:
            resultado.append(ptr)
            ptr = ptr.pai
        return resultado.reverse()

    @staticmethod
    def modelo_transicao(estado):

        indiceLinha = 0
        indiceColuna = 0

        extremoLinha = 20
        extremoColuna = 30
        minimoLinha = 0
        minimoColuna = 0

        vizinhos = []
        visitados = []

        for i, linha in enumerate(estado.vetor):
            for j, coluna in enumerate(linha):
                if linha[j] == 1:
                    indiceLinha = i
                    indiceColuna = j
                    break

        # verifica estado vizinho

        # Ações
        #   ^
        # <- ->
        #   v

        # cima ^
        if (indiceLinha - 1 >= minimoLinha) and (indiceLinha <= extremoLinha) and (indiceColuna >= minimoColuna) and (
                indiceColuna <= extremoColuna):
            estadoAtual = deepcopy(estado.vetor)
            if estadoAtual[indiceLinha - 1][indiceColuna] == 0:
                estadoAtual[indiceLinha][indiceColuna] = 0
                estadoAtual[indiceLinha - 1][indiceColuna] = 1
                ponto_atual = Ponto(indiceColuna + 1, (indiceLinha - 1) + 1)
                ponto_destino = Ponto(20, 31)
                vizinho = Estado(estadoAtual, funcao_avaliacao(ponto_atual, ponto_destino), 'Cima: ^', estado)
                vizinhos.append(vizinho)

        # frente ->
        if (indiceLinha >= minimoLinha) and (indiceLinha <= extremoLinha) and (indiceColuna >= minimoColuna) and (
                indiceColuna + 1 <= extremoColuna):
            estadoAtual = deepcopy(estado.vetor)
            if estadoAtual[indiceLinha][indiceColuna + 1] == 0:
                estadoAtual[indiceLinha][indiceColuna] = 0
                estadoAtual[indiceLinha][indiceColuna + 1] = 1
                ponto_atual = Ponto((indiceColuna + 1) + 1, indiceLinha + 1)
                ponto_destino = Ponto(20, 31)
                vizinho = Estado(estadoAtual, funcao_avaliacao(ponto_atual, ponto_destino), 'Frente: ->', estado)
                vizinhos.append(vizinho)

        # baixo v
        if (indiceLinha >= minimoLinha) and (indiceLinha + 1 <= extremoLinha) and (indiceColuna >= minimoColuna) and (
                indiceColuna <= extremoColuna):
            estadoAtual = deepcopy(estado.vetor)
            if estadoAtual[indiceLinha + 1][indiceColuna] == 0:
                estadoAtual[indiceLinha][indiceColuna] = 0
                estadoAtual[indiceLinha + 1][indiceColuna] = 1
                ponto_atual = Ponto(indiceColuna + 1, (indiceColuna + 1) + 1)
                ponto_destino = Ponto(20, 31)
                vizinho = Estado(estadoAtual, funcao_avaliacao(ponto_atual, ponto_destino), 'Baixo: v', estado)
                vizinhos.append(vizinho)

        # lado esquerdo <-
        if (indiceLinha - 1 >= minimoLinha) and (indiceLinha <= extremoLinha) and (indiceColuna - 1 >= minimoColuna) \
                and (indiceColuna <= extremoColuna):
            estadoAtual = deepcopy(estado.vetor)
            if estadoAtual[indiceLinha - 1][indiceColuna - 1] == 0:
                estadoAtual[indiceLinha][indiceColuna] = 0
                estadoAtual[indiceLinha - 1][indiceColuna - 1] = 1
                ponto_atual = Ponto((indiceColuna - 1) + 1, (indiceLinha - 1) + 1)
                ponto_destino = Ponto(20, 31)
                vizinho = Estado(estadoAtual, funcao_avaliacao(ponto_atual, ponto_destino), 'Trás: <-', estado)
                vizinhos.append(vizinho)

        vizinhos = sorted(vizinhos, key=Estado.get_custo, reverse=True)

        return vizinhos.pop()


def busca_gulosa(problema: Problema):
    """Agente que utiliza a estrategia de busca em largura."""
    print("Usando Busca em Largura: ")
    # 1. Adiciona o estado inicial na lista de borda
    borda = [problema.estado_inicial]  # fringe
    print("Estado Inicial:")

    memoria = [problema.estado_inicial]

    while True:

        # 2. Verifica se houve falha
        if not borda:
            raise RuntimeError('Falha ao encontrar solucao')

        # 3. Recupera o proximo estado
        estado = borda.pop(0)
        print(estado)

        # 4. Verifica se achou a solucao do problema
        if problema.estado_objetivo(estado):
            print('Estado Final encontrado: ')
            print(estado.__repr__())
            return problema.solucao(estado)

        # 5. Geracao dos estados vizinhos
        # -- Adiciona os estados ao final da lista - FIFO - Busca em largura
        vizinhos = problema.modelo_transicao(estado)

        if vizinhos not in memoria:
            borda.append(vizinhos)
            memoria.append(vizinhos)

def main():
    p = Problema()
    busca_gulosa(p)


if __name__ == '__main__':
    main()

ValueError: ignored